In [17]:
import pandas as pd
import json
import random
from datetime import datetime, timedelta

from pymongo import MongoClient
import os

In [2]:
df_cust = pd.read_csv("dataset/customers.csv")
df_cust.head()
# df_cust.shape

,Cust_ID,Cust_firstname,Cust_lastname,Cust_email,Cust_phone_number,Cust_password,Cust_sex,Cust_country,Cust_region,Cust_department,Cust_district,Cust_date_register
0,A000020,Fama,Dieye,famadieye81@gmail.com,221787983005,b'$2b$12$9WcEQOPqZa3F25uXGRFqLuRXywacefdp0Uqo6...,Féminin,Sénégal,Thiès,Thiès,Ngoundiane,2022-03-20
1,A000021,Aicha,Sagna,aichasagna6@gmail.com,221774160499,b'$2b$12$uGFf3PlJtaFSvc.WBRn65uCZmU/c5oiatpaop...,Féminin,Sénégal,Kaolack,Kaolack,Ndiaffate,2022-03-20
2,A000022,Abdou,Diamanka,abdoudiamanka@gmail.com,221771545961,b'$2b$12$veizFY0Nsg7lySk3pjUeAOr0tgomFoZ9oqtiH...,Masculin,Sénégal,Dakar,Dakar,Hlm,2022-03-22
3,A000023,Abdoulaye,Sam,abdoulayesam92@gmail.com,221776361782,b'$2b$12$WHsalQG4w5ZG15pwmIBrxufIqgGqX2wv/VAsm...,Masculin,Sénégal,Dakar,Guédiawaye,Ndiareme,2022-03-23
4,A000024,Lamine,Niang,lamineniang@gmail.com,221786288713,b'$2b$12$M28DMsaXx.iEh45VW/h7x.WZ9rHPpRs8kNpLw...,Masculin,Sénégal,Dakar,Guédiawaye,Medina Gounass,2022-03-23


In [3]:
df_products = pd.read_csv("dataset/products.csv")
df_products.head() #1286
# df_products[df_products["Product_ID"] == "P000006"]

,Product_ID,Category_ID,Product,Product_name,Description_Product,Price,Currency
5,P000006,Cat_0002,7999,Becca Aqua Luminous Perfecting Foundation Fond...,Teint,27000,FCFA


In [4]:
df_orders = pd.read_csv("dataset/orders.csv")
df_orders.head()

,Cust_ID,Shipping_cost_ID,Number_items,delivery_address,Order_date,Shipped_date,Delivery_date,Delivery_formula,Payment_channel,Total_amount,Total_amount_invoiced,Order_ID
0,A000021,Fee_070,3,Thiomby,2022-04-17,2022-04-22,2022-04-26,Standard,Orange-Money,441000,441000.0,O000001
1,A000025,Fee_004,5,Medina,2022-04-21,2022-04-25,2022-04-28,Standard,Wave,554500,554500.0,O000002
2,A000040,Fee_017,5,Grand Yoff,2022-04-25,2022-04-28,2022-05-02,Standard,Orange-Money,356600,356600.0,O000003
3,A000040,Fee_012,1,Sicap Liberte,2022-05-01,2022-05-02,2022-05-04,Express,Orange-Money,70500,70500.0,O000004
4,A000052,Fee_077,5,Sessene,2022-05-22,2022-05-24,2022-05-26,Express,Wave,239900,239900.0,O000005


In [5]:
df_order_details = pd.read_csv("dataset/order_details.csv")
df_order_details.head()

,Order_details_ID,Order_ID,Product_ID,Promotion_ID,Order_line_number,Quantity_ordered,Unity_price,Amount_net,Amount_reduced,Amount_invoiced
0,Od008973,O000001,P001057,NaN,1,4,21000,84000,0.0,84000.0
1,Od008974,O000001,P000603,NaN,2,4,36000,144000,0.0,144000.0
2,Od008975,O000001,P000243,NaN,3,3,70000,210000,0.0,210000.0
3,Od008976,O000002,P000733,NaN,1,3,15500,46500,0.0,46500.0
4,Od008977,O000002,P000117,NaN,2,3,18000,54000,0.0,54000.0


In [6]:
# quelques opérations

# recherche de la quantité totale acheté par produit
sum_quantities = df_order_details.groupby('Product_ID')['Quantity_ordered'].sum() # une série

sum_quantities_df = sum_quantities.reset_index() # je convertis en dataframe

sum_quantities_df.columns = ['Product_ID', 'Total_Quantity']

sum_quantities_df_sorted = sum_quantities_df.sort_values(by='Total_Quantity', ascending=False)

quantity_distinct = sum_quantities_df_sorted["Total_Quantity"].unique()


def check_product(sum_quantities_df, quantity) : 
    list_product_ID = sum_quantities_df[sum_quantities_df["Total_Quantity"] == quantity]
    return list_product_ID["Product_ID"].tolist()

def check_order(df_order_details, product_id) : 
    list_orders = df_order_details[df_order_details["Product_ID"] == product_id]
    list_orders = list_orders["Order_ID"].tolist()
    return list_orders

def check_customer(df_orders, list_orders):
    lst_cust = []
    for i in list_orders :
        cust_id = df_orders[df_orders["Order_ID"] == i]
        cust_id = cust_id['Cust_ID'].iloc[0]
        lst_cust.append(cust_id)
    return lst_cust

def random_start_end_time():
    lst_time= []
    random_hour = random.randint(9, 23)
    random_minute = random.randint(0, 59)
    random_second = random.randint(0, 59)

    random_time = datetime(2024, 3, 20, random_hour, random_minute, random_second)

    viewing_time = random.randint(10, 80)
    new_time = random_time + timedelta(seconds=viewing_time)

    start_time = random_time.strftime("%H:%M:%S")
    end_time =  new_time.strftime("%H:%M:%S")
    
    lst_time.append(start_time)
    lst_time.append(end_time)
    lst_time.append(random_hour)
    
    return lst_time

def navigation (df_cust, df_orders, df_order_details, list_product_ID):
    lst_browser_used = ["Google Chrome", "Mozilla Firefox", "Safari", "Microsoft Edge", "Opera", "Brave", "Internet Explorer"]
    lst_os_used = ["Windows", "macOS", "iOS", "Android"]
    lst_device = ["Ordinateur portable", "Smartphone", "Tablette", "Samsung", "Iphone", "Tecno"]
    lst_action = ["Viewing", "Cart"]
    lst_nav = []
    for p in list_product_ID :
        number_of_customers = random.choice([80, 90, 75, 120])
        random_cust_ids = random.sample(list(df_cust['Cust_ID'].unique()), number_of_customers) #list_cust
        list_orders = check_order(df_order_details, p)
        lst_cust_orders = check_customer(df_orders, list_orders)
        already_purchased = []
        count = random.choice([70, 80, 80, 100, 130, 150])
        for nav in range(count):
            dic_nav = {}
            browser_used = random.choice(lst_browser_used)
            os_used = random.choice(lst_os_used)
            device = random.choice(lst_device)
            type_action = random.choice(lst_action)
            info_time = random_start_end_time()
            viewing_start_time = info_time[0]
            viewing_end_time = info_time[1] 
            hour_visited = info_time[2] 
            
            random_cust_ids.extend(lst_cust_orders)
            cust_id =  random.choice(random_cust_ids)
            if (cust_id in lst_cust_orders and cust_id not in already_purchased) :
                type_action = "Buy"
                already_purchased.append(cust_id)
            
            dic_nav["cust_id"] = cust_id
            dic_nav["product_id"] = p
            dic_nav["viewing_start_time"] = viewing_start_time
            dic_nav["viewing_end_time"] = viewing_end_time
            dic_nav["hour_visited"] = hour_visited
            dic_nav["type_action"] = type_action
            dic_nav["browser_used"] = browser_used
            dic_nav["os_used"] = os_used
            dic_nav["device"] = device
            
            lst_nav.append(dic_nav)
            
    return lst_nav
    

In [7]:
quantity_distinct

array([26, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,
        8,  7,  6,  5,  4,  3,  2,  1], dtype=int64)

In [13]:
# appel de la fonction navigation
all_navigation = []
for q in quantity_distinct :
    list_product_ID = check_product(sum_quantities_df, q)  # list des produits
    print("Nombre de P : " , len(list_product_ID))
    nav_current = navigation (df_cust, df_orders, df_order_details, list_product_ID)  # la sortie est une liste 
    print("Nombre d'enregistrement de navigation : ", len(nav_current))
    all_navigation.extend(nav_current)

Nombre de P :  4
Nombre d'enregistrement de navigation :  360
Nombre de P :  3
Nombre d'enregistrement de navigation :  360
Nombre de P :  4
Nombre d'enregistrement de navigation :  330
Nombre de P :  5
Nombre d'enregistrement de navigation :  440
Nombre de P :  10
Nombre d'enregistrement de navigation :  960
Nombre de P :  6
Nombre d'enregistrement de navigation :  560
Nombre de P :  7
Nombre d'enregistrement de navigation :  800
Nombre de P :  13
Nombre d'enregistrement de navigation :  1430
Nombre de P :  29
Nombre d'enregistrement de navigation :  2760
Nombre de P :  36
Nombre d'enregistrement de navigation :  3750
Nombre de P :  41
Nombre d'enregistrement de navigation :  4390
Nombre de P :  52
Nombre d'enregistrement de navigation :  5360
Nombre de P :  57
Nombre d'enregistrement de navigation :  5970
Nombre de P :  88
Nombre d'enregistrement de navigation :  8700
Nombre de P :  87
Nombre d'enregistrement de navigation :  8950
Nombre de P :  89
Nombre d'enregistrement de navigati

In [14]:
len(all_navigation) # 127400

127400

In [16]:
all_navigation.head()

[{'cust_id': 'A000557',
  'product_id': 'P000006',
  'viewing_start_time': '15:50:46',
  'viewing_end_time': '15:51:12',
  'hour_visited': 15,
  'type_action': 'Cart',
  'browser_used': 'Brave',
  'os_used': 'Windows',
  'device': 'Iphone'},
 {'cust_id': 'A000212',
  'product_id': 'P000006',
  'viewing_start_time': '13:46:25',
  'viewing_end_time': '13:46:51',
  'hour_visited': 13,
  'type_action': 'Cart',
  'browser_used': 'Internet Explorer',
  'os_used': 'macOS',
  'device': 'Smartphone'},
 {'cust_id': 'A000251',
  'product_id': 'P000006',
  'viewing_start_time': '12:39:16',
  'viewing_end_time': '12:39:51',
  'hour_visited': 12,
  'type_action': 'Cart',
  'browser_used': 'Safari',
  'os_used': 'Android',
  'device': 'Iphone'},
 {'cust_id': 'A000325',
  'product_id': 'P000006',
  'viewing_start_time': '19:35:48',
  'viewing_end_time': '19:36:41',
  'hour_visited': 19,
  'type_action': 'Buy',
  'browser_used': 'Internet Explorer',
  'os_used': 'macOS',
  'device': 'Iphone'},
 {'cust_

In [23]:
# Définition du chemin du fichier JSON
file_navigation = "dataset/navigation.json"
if os.path.exists(file_navigation):
    print("existe")
    os.remove(file_navigation)

with open(file_navigation, "w") as fichier_json:
    print("let's go")
    random.shuffle(all_navigation)
    json.dump(all_navigation, fichier_json, indent=4)  # Utilisez fichier_json ici au lieu de file_navigation

print("Les données ont été écrites dans", file_navigation)



existe
let's go
Les données ont été écrites dans dataset/navigation.json


In [24]:
file_navigation = "dataset/navigation.json"

# Connexion à la base de données MongoDB
client = MongoClient('localhost', 27017)
db = client.dbProductElunea  # Remplacez 'ma_base_de_donnees' par le nom de votre base de données
collection = db.collection_navigations  # Remplacez 'ma_collection' par le nom de votre collection

# Lecture du fichier JSON et insertion dans la collection MongoDB
with open(file_navigation, "r") as fichier_json:
    donnees_json = json.load(fichier_json)
    collection.insert_many(donnees_json)

print("Les données ont été chargées dans la collection MongoDB avec succès.")


Les données ont été chargées dans la collection MongoDB avec succès.
